In [ ]:
from openai import OpenAI

In [ ]:
client = OpenAI()

## Q1 Running Elastic

- Getting data
- Indexing data
- Searching
- Filtering
- Build a prompt

### Getting data

In [ ]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Indexing data

In [ ]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

index_name = "hw"
es = Elasticsearch('http://localhost:9200')  # Connect to your local Elasticsearch instance

# create Index with es
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} # keyword for filtering
        }
    }
}

index_name = 'hw-llm'
es.indices.create(index=index_name, body=index_settings)

In [ ]:
for doc in tqdm(documents):
    es.index(index=index_name, body=doc) 

### Searching

In [ ]:
query = "How do I execute a command in a running docker container?"
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
            }
        }
    }

response = es.search(index=index_name, body=search_query)

In [ ]:
top_score = response['hits']['hits'][0]['_score']

In [ ]:
top_score

### Filtering

In [ ]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], # question are three times more important
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": 'machine-learning-zoomcamp'
                }
            }
        }
    }
}

In [ ]:
response = es.search(index=index_name, body=search_query)

result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [ ]:
result_docs[2]

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context_template = """
                Q: {question}
                A: {text}
                """.strip()
                    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt